In [86]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump

In [72]:
# CSVファイルを読み込む
df = pd.read_csv('scraped_data_forML.csv', index_col=0)

df.head(2)

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市昭和区滝子通４,地下鉄桜通線/桜山駅 歩7分,地下鉄桜通線/瑞穂区役所駅 歩15分,地下鉄鶴舞線/荒畑駅 歩15分,築8年,2階建,1階,5.3万円,4000円,-,5.3万円,1SK,20.9m2,https://suumo.jp/chintai/jnc_000088214111/?bc=...,2024-02-19
1,賃貸マンション,フロイデ桜山,愛知県名古屋市昭和区菊園町４,地下鉄桜通線/桜山駅 歩11分,地下鉄鶴舞線/川名駅 歩17分,地下鉄桜通線/瑞穂区役所駅 歩19分,築6年,3階建,1階,8万円,4500円,-,8万円,1LDK,46.84m2,https://suumo.jp/chintai/jnc_000088573366/?bc=...,2024-02-19


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2106
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   property_type    1994 non-null   object
 1   building_name    1994 non-null   object
 2   address          1994 non-null   object
 3   access_1         1994 non-null   object
 4   access_2         1980 non-null   object
 5   access_3         1934 non-null   object
 6   age              1994 non-null   object
 7   building_floors  1994 non-null   object
 8   room_floor       1994 non-null   object
 9   rent             1994 non-null   object
 10  management_fee   1994 non-null   object
 11  deposit          1994 non-null   object
 12  gratuity         1994 non-null   object
 13  layout           1994 non-null   object
 14  area             1994 non-null   object
 15  url              1994 non-null   object
 16  posting_date     1994 non-null   object
dtypes: object(17)
memory usage: 280.4

# 前処理
1. 実質負担額を作成する
   1. rent +  management_fee +  (deposit / 48)
2. access_1, access_2, access_3から最寄駅ではなくて桜山駅からの時間を取得する
2. 数値型に変換する
   1. age, building_floors, room_floor
   2. area

In [74]:
# 共益費management_feeと敷金(deposit)と礼金(gratuity)が「-」の場合は0とする
df['management_fee'] = df['management_fee'].replace('-', '0円')
df['deposit'] = df['deposit'].replace('-', '0万円')
df['gratuity'] = df['gratuity'].replace('-', '0万円')

# 金額関連の列を数値に変換する関数
def yen_to_float(yen_str):
    return float(yen_str.replace('万円', '')) * 10000

# rent, management_fee, deposit, gratuity を数値に変換
df['rent'] = df['rent'].apply(yen_to_float)
df['management_fee'] = df['management_fee'].str.replace('円', '').astype(float)
df['deposit'] = df['deposit'].apply(yen_to_float)
df['gratuity'] = df['gratuity'].apply(yen_to_float)

# 面積を数値に変換
df['area'] = df['area'].str.replace('m2', '').astype(float)

# 実質負担額を計算（rent + management_fee + (deposit + gratuity) / 48）
df['actual_cost'] = df['rent'] + df['management_fee'] + (df['deposit'] + df['gratuity']) / 48

In [75]:
# 桜山駅からの最短時間を取得する関数
def get_min_time_to_sakurayama(access_str):
    if pd.isnull(access_str):
        return np.nan
    times = [int(s.split('歩')[1].replace('分', '')) for s in access_str.split('/') if '桜山駅' in s]
    if times:
        return min(times)
    else:
        return np.nan

# access_1, access_2, access_3から桜山駅までの最短時間を計算
df['min_time_to_sakurayama'] = df[['access_1', 'access_2', 'access_3']].apply(
    lambda x: min(
        filter(pd.notnull, [get_min_time_to_sakurayama(x['access_1']), get_min_time_to_sakurayama(x['access_2']), get_min_time_to_sakurayama(x['access_3'])]),
        default=np.nan
    ),
    axis=1
)

In [76]:
# '築新'または'新'を'0'年として扱うための処理を追加
df['age'] = df['age'].str.replace('築新', '0').str.replace('新', '0').str.replace('築', '').str.replace('年', '')


# 空の値を0に置き換え
df['age'] = df['age'].replace('', '0')

# 整数型に変換
df['age'] = df['age'].astype(int)

# 建物の階数と部屋の階についても同様に処理
df['building_floors'] = df['building_floors'].str.extract('(\d+)')[0].fillna('0').astype(int)
df['room_floor'] = df['room_floor'].str.extract('(\d+)')[0].fillna('0').astype(int)

# ここでのfillna('0')は、抽出した結果がNaN（該当する数値がない場合）の場合に0を割り当てるために使用しています。

In [77]:
df.head()

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date,actual_cost,min_time_to_sakurayama
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市昭和区滝子通４,地下鉄桜通線/桜山駅 歩7分,地下鉄桜通線/瑞穂区役所駅 歩15分,地下鉄鶴舞線/荒畑駅 歩15分,8,2,1,53000.0,4000.0,0.0,53000.0,1SK,20.90,https://suumo.jp/chintai/jnc_000088214111/?bc=...,2024-02-19,58104.166667,7
1,賃貸マンション,フロイデ桜山,愛知県名古屋市昭和区菊園町４,地下鉄桜通線/桜山駅 歩11分,地下鉄鶴舞線/川名駅 歩17分,地下鉄桜通線/瑞穂区役所駅 歩19分,6,3,1,80000.0,4500.0,0.0,80000.0,1LDK,46.84,https://suumo.jp/chintai/jnc_000088573366/?bc=...,2024-02-19,86166.666667,11
2,賃貸マンション,アヴニール桜山,愛知県名古屋市昭和区桜山町５,地下鉄桜通線/桜山駅 歩2分,地下鉄鶴舞線/御器所駅 歩12分,地下鉄鶴舞線/荒畑駅 歩18分,4,8,3,131000.0,9000.0,0.0,196500.0,2LDK,68.67,https://suumo.jp/chintai/jnc_000088573368/?bc=...,2024-02-19,144093.750000,2
3,賃貸マンション,Ezサイト,愛知県名古屋市昭和区大和町１,地下鉄桜通線/桜山駅 歩6分,地下鉄鶴舞線/御器所駅 歩8分,地下鉄鶴舞線/荒畑駅 歩16分,22,3,3,170000.0,0.0,340000.0,170000.0,3SLDK,77.96,https://suumo.jp/chintai/jnc_000076484377/?bc=...,2024-02-19,180625.000000,6
4,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,8,2,1,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000088370773/?bc=...,2024-02-19,58614.583333,3


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2106
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_type           1994 non-null   object 
 1   building_name           1994 non-null   object 
 2   address                 1994 non-null   object 
 3   access_1                1994 non-null   object 
 4   access_2                1980 non-null   object 
 5   access_3                1934 non-null   object 
 6   age                     1994 non-null   int64  
 7   building_floors         1994 non-null   int64  
 8   room_floor              1994 non-null   int64  
 9   rent                    1994 non-null   float64
 10  management_fee          1994 non-null   float64
 11  deposit                 1994 non-null   float64
 12  gratuity                1994 non-null   float64
 13  layout                  1994 non-null   object 
 14  area                    1994 non-null   

## object型の処理
- エンコーディングする
  - property_type
  - address
  - layout
- dropする
  - building_name
  - access_1
  - access_2
  - access_3
  - url
  - posting_date

In [79]:
# エンコーディングするカラム
categorical_cols = ['property_type', 'address', 'layout']

# One-hotエンコーディング
df = pd.get_dummies(df, columns=categorical_cols)

# 不要なカラムを削除
drop_cols = ['building_name', 'access_1', 'access_2', 'access_3', 'url', 'posting_date']
df.drop(drop_cols, axis=1, inplace=True)

In [80]:
df.head(2)

,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,area,actual_cost,min_time_to_sakurayama,...,layout_3DK,layout_3K,layout_3LDK,layout_3SDK,layout_3SLDK,layout_4DK,layout_4LDK,layout_5LDK,layout_5SLDK,layout_ワンルーム
0,8,2,1,53000.0,4000.0,0.0,53000.0,20.90,58104.166667,7,...,0,0,0,0,0,0,0,0,0,0
1,6,3,1,80000.0,4500.0,0.0,80000.0,46.84,86166.666667,11,...,0,0,0,0,0,0,0,0,0,0


# モデルの訓練

In [87]:
# 特徴量とターゲット変数に分割
X = df.drop('actual_cost', axis=1)  # actual_cost以外のすべてのカラムを特徴量として使用
y = df['actual_cost']  # actual_costをターゲット変数として使用

# データセットの分割（例：訓練データ80%、テストデータ20%）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
# モデルの訓練とテスト
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

for name, model in models.items():
    # モデルの訓練
    model.fit(X_train, y_train)
    
    # テストデータで予測
    predictions = model.predict(X_test)
    
    # RMSEの計算
    rmse_score = np.sqrt(mean_squared_error(y_test, predictions))
    print(f"{name}: Test RMSE: {rmse_score:.4f}")
    
    # モデルの保存
    dump(model, f'{name.replace(" ", "_").lower()}_model.joblib')

Random Forest: Test RMSE: 1306.0266
Gradient Boosting: Test RMSE: 953.7290
